# import packages

In [1]:
from sys import exit
import pandas as pd
import numpy as np
import csv
from urllib import request
from sodapy import Socrata
from io import StringIO

# important functions

In [2]:
#if value is NaN, return ifnan_value, otherwise return value
def if_null_value(value,if_nan_value):
    return (if_nan_value if pd.isnull(value) else value)

# If state is listed as "ZZ" or "XX", than zip codeis null.
# Fill with 1 if 'ZZ' (unknown location)
# or 2 if 'XX'. Not in the US 
# or 3 if 'AP' for Pacific armed forces
def fill_zip_code(state,zipcode):
    final_code = ''
    if not pd.isnull(zipcode):
        final_code = zipcode
    elif state == 'ZZ':
        final_code = 1
    elif state == 'XX':
        final_code = 2
    elif state == 'AP':
        final_code = 3
    else:
        print('ERROR: state: \'{}\' not recognized'.format(state))
        exit()
    return final_code


#combines strings, used to combine first and last names with spaces between.
def combine_strings(*strings):
    combine = ''
    for x in strings:
        if isinstance(x,str):
          combine += x + ' '
    if(combine == ''):
        return combine
    else:
        return combine[0:-1] 
    

# Create dataframe of population by zipcode
Dowloaded from kaggle. A database to link the population to zipcode is created from the 2010 census 

Source: https://www.kaggle.com/census/us-population-by-zip-code
Accessed: October 30, 2018

Several zipcode have empty populations. Using a second database that links zipcode to city, the population 
of these zipcodes will be estimated from average zip code in the same city. The csv file for which this database is create does contain population of zip codes. Because their source of their population is not cited, it will not be used.

Source of second database: https://simplemaps.com/data/us-zips 
accessed: November 7, 2018

In [3]:
#download population from kaggle
!kaggle datasets download -d census/us-population-by-zip-code
!unzip us-population-by-zip-code.zip
!mkdir population_kaggle/
!mv population_by_zip_20* population_kaggle/

#data lists population organized by gender and range age. It also lists the total population 
# regardless of gender or population. Removing all but the total population of a zip code.
filename = 'population_kaggle/population_by_zip_2010.csv'
population_df = pd.read_csv(filename).fillna(value='all')
population_df = population_df[(population_df['gender'] == 'all') 
              & (population_df['minimum_age'] == 'all') 
              & (population_df['maximum_age'] == 'all')][['population','zipcode']]
#population_df.set_index('zipcode',inplace=True)

#Create a database to link zip codes to cities and states. Average the population zip codes in the cities and states,
# use to find estimate to population of zip codes with unknown/undefine population.
filename = 'Zip_code_to_address/uszipsv1.4.csv'
location_df = pd.read_csv(filename)[['zip','city','state_id']]
#location_df.set_index('zip',inplace=True)
population_df = population_df.merge(location_df,how='right',left_on='zipcode',right_on='zip')
population_df.drop(columns='zipcode',inplace=True)
population_df.rename(columns={'zip':'zipcode'},inplace=True)

#Average zip codes of the same city and state.
#Fills zip codes with null population with  average
location_pop=population_df.groupby(['state_id','city'])['city','state_id','population'].mean().reset_index()
population_df= population_df.merge(location_pop,how='left',on=['state_id','city'])
population_df.rename(columns={'population_x':'population','population_y':'population avg'},inplace=True)
population_df['population'] = population_df[['population','population avg']].apply(lambda x:if_null_value(x[0],x[1]),axis=1)
population_df.drop(columns='population avg',axis=1,inplace=True)

us-population-by-zip-code.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  us-population-by-zip-code.zip
  inflating: population_by_zip_2000.csv  
  inflating: population_by_zip_2010.csv  
mkdir: cannot create directory ‘population_kaggle/’: File exists


# Create dataframe for API

Here a dataframe for the percentage and amount of opioid prescribed using Medicare Part D by health care providers. The data is cleaned and using the previous dataset, the population of the

Source: https://data.cms.gov/Medicare-Claims/Medicare-Part-D-Opioid-Prescriber-Summary-File-201/yb2j-f3fp

In [4]:
WEBSITE = 'data.cms.gov'
DATASET_ID = 'aksg-4qws'

client = Socrata(WEBSITE,None)
results = client.get(DATASET_ID,content_type='json',limit=1500000)
opioid_df = pd.DataFrame.from_records(results)[['npi',
                                                'nppes_provider_first_name',
                                                'nppes_provider_last_name',
                                                'nppes_provider_zip_code',
                                                'nppes_provider_state',
                                                'specialty_description',
                                                'total_claim_count',
                                                'opioid_claim_count',
                                                'percent_opioid_claims']]


#change name of columns
opioid_df.columns =['doctor id',
                    'first name',
                    'last name',                    
                    'zip code',                       
                    'state',
                    'specialty description',
                    'total claims',
                    'opioid claims',
                    'fraction opioid claims']
#set doctor id to the index
opioid_df.set_index('doctor id',inplace=True)
#combine first and last name into a single column
opioid_df['doctor name']=opioid_df[['first name','last name']].apply(lambda x:combine_strings(*x),axis=1)
opioid_df.drop(labels=['first name','last name'],axis=1,inplace=True)

#Unknown location (state listed as ZZ) gets a zipcode of 0
#Not in the US (state listed as XX) get a zipcode of 1
opioid_df['zip code'] = opioid_df[['state','zip code']].apply(lambda x:fill_zip_code(x[0],x[1]),axis=1)


#setting specialty description index
doc_spec = set(list(opioid_df['specialty description']))
spec_compreh=((x,counter) for counter, x in enumerate(doc_spec))
spec_df=pd.DataFrame(spec_compreh,columns=['specialty description','specialty index'])
opioid_df = opioid_df.merge(spec_df,how='left',on='specialty description')

#merging with population data
population_df['zipcode'] = population_df['zipcode'].astype(int)
opioid_df['zip code'] = opioid_df['zip code'].astype(int)
opioid_df = opioid_df.merge(population_df,how='left',left_on='zip code',right_on='zipcode')
opioid_df.drop('zipcode',axis=1,inplace=True)

#Droping doctors with nan or zero claims  
opioid_df['opioid claims']= opioid_df['opioid claims'].apply(lambda x: if_null_value(x,0))
opioid_df['opioid claims']= opioid_df['opioid claims'].astype(int)
opioid_df['fraction opioid claims']=opioid_df['fraction opioid claims'].apply(lambda x: if_null_value(x,0))
opioid_df['fraction opioid claims']=opioid_df['fraction opioid claims'].astype(float)
opioid_reduced=opioid_df[opioid_df['opioid claims'] != 0.0]


# Quick analyze of data and Outlier detection

In [5]:
print('# health providers: {}'
      .format(opioid_df['doctor name'].count()))
print('# health providers who prescribe opioid: {}'
      .format(opioid_reduced['doctor name'].count()))
print('% health providers that prescribe opioids: {:.2f}%'.
     format(opioid_reduced['doctor name'].count()/opioid_df['doctor name'].count()*100))
print('# specialities: {}'.format(spec_df['specialty index'].count()))
print('# specialities that prescribe opioid: {}'
      .format(opioid_reduced['specialty index'].nunique()))
print('Max % prescriptions that are opioids: {:.2f}%'
      .format(opioid_reduced['fraction opioid claims'].max()*100))
print('Min % prescriptions that are opioids(other than zero): {:.2f}%'
      .format(opioid_reduced['fraction opioid claims'].min()*100))
print('Missing population for {} ({:.2f}%) entries'.
      format(opioid_df[(opioid_df['population'].isnull())]['zip code'].count()
          ,opioid_df[(opioid_df['population'].isnull()) ]['zip code'].count()/opioid_df['zip code'].count()*100))


# health providers: 1049326
# health providers who prescribe opioid: 496744
% health providers that prescribe opioids: 47.34%
# specialities: 246
# specialities that prescribe opioid: 169
Max % prescriptions that are opioids: 100.00%
Min % prescriptions that are opioids(other than zero): 0.03%
Missing population for 1836 (0.17%) entries


# Get rid of the remaing null values

In [6]:
opioid_df=opioid_df[opioid_df['population'].notnull()]
opioid_reduced=opioid_reduced[opioid_reduced['population'].notnull()]